<a href="https://colab.research.google.com/github/antoinechatain/Chiffre-d-affaires-d-Adidas---Series-Temporelles/blob/main/BigData.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#TP sur SPARK
!pip install pyspark


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
#On instancie Spark
from pyspark.sql import SparkSession
spark = SparkSession.builder\
        .master("local")\
        .appName("Colab")\
        .config('spark.ui.port','4050')\
        .getOrCreate()

In [ ]:
#On importe le fichier 
sc = spark.sparkContext
book = sc.textFile("/content/book.txt")
book

/content/book.txt MapPartitionsRDD[1] at textFile at NativeMethodAccessorImpl.java:0

In [ ]:
#On teste les méthodes. La méthode take prend les n premiers, ici 20 car l'arguement est 20
book.take(20)

In [ ]:
# On teste les attributs 

book.first

<bound method RDD.first of /content/book.txt MapPartitionsRDD[1] at textFile at NativeMethodAccessorImpl.java:0>

In [ ]:
#D'autres attributs

book.count

<bound method RDD.count of /content/book.txt MapPartitionsRDD[1] at textFile at NativeMethodAccessorImpl.java:0>

In [ ]:
# On filtre les lignes selon des conditions que l'on choisit (200 est demandé)

filtered = book.filter(lambda x: len(x) >= 200).collect()
filtered
len(filtered)

In [ ]:
# On utilise la méthode flatMap pour décomposer chaque ligne en mot

words = book.flatMap(lambda x : x.split(" "))

In [ ]:
#extraction pour l'aperçu
words.take(20)
words_one = words.map(lambda x : (x,1))
words_one.take(5)

In [ ]:
words_one = words.map(lambda x : (x,1))
words_one.take(4)
#On utilise la méthode ReduceByKey pour compter le nombre de fois qu'un mot apparait
words_count = words_one.reduceByKey(lambda x,y : x+y)
words_count

In [ ]:
# On filtre le le mot "someone"
for (i,j) in words_count:
  if i =="someone":
    print(i,j)
w = words_count.filter(lambda x: x[0]=="someone")
w.take(3)

In [ ]:
# On trie par le nombre de fois que le mot apparaît
inverse = words_count.map(lambda x: (x[1],x[0]))
sorted.map(lambda x: (x[1],x[0]))

In [ ]:
sorted.take(2300)

In [ ]:
#  EXERCICE Fake friends
# Depuis le module pyspark.sql, on importe functions sous l'alias F

from pyspark.sql import functions as F


In [ ]:
#On lit le fichier fakefriends sous le bon format
data = spark.read.format("csv").load("fakefriends.txt").schema(schema)

In [ ]:
#On regarde le type des colonnes
type(data)

In [ ]:
# On renomme maintenant les colonnes (leur en-tête)
data_renamed= data.withColumnRenamed("c_1","Nom").withColumnRenamed("c_2","Valeur 1").withColumnRenamed("c_3","Valeur 2").rdd
type(data_renamed)

In [ ]:
# On instancie
from pyspark.sql.types import StructType,StructField, StringType, IntegerType
from pyspark.sql.types import IntegerType


schema = StructType([ \
    StructField("Index",StringType(),True),\
    StructField("lastName",StringType(),True), \
    StructField("Age",IntegerType(),True), \
    StructField("Number",IntegerType(),True), \
    
  ])


In [ ]:
#On lit le fichier en question au bon format
data = spark.read.format("csv").schema(schema).load("fakefriends (1).txt")

In [ ]:
#On applique des méthodes
data.show(3)
data.take(5);

In [ ]:
#On applique des attributs
data.count
data.first
data.show

In [ ]:
data.select("lastName").show()

In [ ]:
#REPONSES AUX QUESTIONS
# 1) Combien de personnes ont 34 ans ?
#A l'aide d'une requete SQL
data.createOrReplaceTempView("friendsTable")
spark.sql("SELECT * FROM friendsTable").show(5)
spark.sql("SELECT * FROM friendsTable WHERE age = 34").show()

In [ ]:
# 2) Faire un df avec le nombre d'amis total par nom
#Encore une fois, grâce au SQL
DF_nbre_amis = spark.sql("SELECT lastName, sum(Number) FROM friendsTable group by lastName") 
type(DF_nbre_amis)
data.createOrReplaceTempView("friendsTableTotal")
nbre = spark.sql("select mean(Number) FROM friendsTableTotal where age=34").collect()
nbre

In [ ]:
#TP KMEANS
#On importe SparkSession du module pyspark.sql
from pyspark.sql import SparkSession

spark = SparkSession.builder.master("local[*]") \
            	.appName("tp") \
            	.getOrCreate()

In [ ]:
#On lit le fichier avec les bonnes options
coordinate = spark.read.format("csv").option("header", value = True).option("delimiter",";").option("inferSchema",True).load("simple_data_Kmeans.csv")

In [ ]:
#On applique des méthodes
coordinate.printSchema
coordinate.show(10)
coordinate.first()


In [ ]:
#On crée les nuages de points
#On importe le module suivant

import matplotlib.pyplot as plt

coordinate.toPandas().plot.scatter(x='X',y='Y')
plt.plot()


from pyspark.ml.feature import VectorAssembler

assembler = VectorAssembler().setInputCols(["X","Y"]).setOutputCol("features")
coordinate_prepared = assembler.transform(coordinate)
coordinate_prepared.printSchema()
coordinate_prepared.show(5)

In [ ]:
#On divise le dataset en vue du machine learning
training_set,test_set = coordinate_prepared.randomSplit([0.8, 0.2])

In [ ]:
#On exécute l'algorithme du K-means (cluster)
#On importe KMeans
from pyspark.ml.clustering import KMeans
kmeans = KMeans().setK(3).setFeaturesCol("features").setPredictionCol("prediction")

In [ ]:
kmeans_3_model = kmeans.fit(training_set) # entrainement
kmeans_3_model.clusterCenters() # resultats

In [ ]:
#On le trace (avec les options de couleur, de forme que l'on souhaite)
import matplotlib.pyplot as plt
coordinate.toPandas().plot.scatter(x='X',y='Y')
for element in kmeans_3_model.clusterCenters():
	plt.plot([element[0]],[element[1]], marker='*', markerfacecolor='orange', ls='none', ms=20)
plt.show()

In [ ]:
#On regarde l'efficacité du modèle
from pyspark.ml.evaluation import ClusteringEvaluator
predictions = kmeans_3_model.transform(training_set)
evaluator = ClusteringEvaluator()
evaluator.evaluate(predictions) # -1<evaluator.evaluate(predictions)<1

In [ ]:
prediction = kmeans_3_model.transform(test_set)
prediction.show(100)
prediction.groupBy("prediction").count().show()

In [ ]:
from pyspark.ml.clustering import KMeansModel
kmeans_model_loaded = KMeansModel.load("/home/ia/ spark_files /first-model") //Importer

In [ ]:
# Exercice 1

age = 23
name = "Antoine"
size = 1,73



print("bonjour je m'appelle", name, " j'ai ",age," ans et je fais", size, "m" )

bonjour je m'appelle Antoine  j'ai  23  ans et je fais (1, 73) m


In [ ]:
import sys
size_name = sys.getsizeof(name)
size_name

In [ ]:
new_var= str(age)+name
new_var

In [ ]:
sys.getsizeof(new_var)

In [ ]:
#Comparaison
size_age = sys.getsizeof(age)
size_taille = sys.getsizeof(size)
if size_age < size_taille:
  print("taille plus grand age")
else:
  print("age plus grand que taille")

In [ ]:
#Comparaison
student =True
size_age = sys.getsizeof(age)
size_student = sys.getsizeof(student)
if size_age < size_student:
  print("student plus grand age")
else:
  print("age plus grand que student")

In [ ]:
# Exercice 2:

class Person:
  
  def __init__(self, first_name, last_name):
      self.first_name = first_name
      self.last_name = last_name

  
def move(self):
    return f"{self.first_name} is moving..."

In [ ]:
personne = Person("Antoine","Chatain")

In [ ]:
class Student:

  def __init__(self, Person, school_year, class_year):
      self.Person = Person
      self.school_year = school_year
      self.class_year = class_year

In [ ]:
first_student = Student(Person("Antoine","Chatain"), 7, 8)

In [ ]:
#On importe le module re
import re
#On crée une classe
class mail: 
  def __init__(self, email, is_confirmed):
      self.email = email
      self.is_confirmed = is_confirmed
    
  def extraction(self):
    partie_mail = self.email.split("@",2)
    print(partie_mail[1])

In [ ]:
first_mail = mail("chatainant@gmail.com", True)
first_mail.extraction()

In [ ]:
# On ajoute la personne ABC

from abc import ABC

class Person(ABC):
  
  
  def __init__(self, email,first_name, last_name):
      self.email = mail.email
      self.first_name = first_name
      self.last_name = last_name

In [ ]:
# Exercice 3 :

#On import numpy

import numpy as np 


In [ ]:
#Création d'un vecteur
vect1 = np.zeros(10)

In [ ]:
#On utilise un attribut
vect1.shape

In [ ]:
#On rentre une valeur
vect1[4] = 5

In [ ]:
vect1_reverse = vect1[::-1]

In [ ]:
vect1_deleted = np.delete(vect1,len(vect1)-1)
vect1_deleted

In [ ]:
#On crée une matrice
matrice = np.reshape(vect1_deleted,(3,3))
matrice

In [ ]:
#On rentre une valeur
matrice[2,2] = 9

In [ ]:
#On utilise des méthodes
matrice.max()
matrice.min()
matrice.mean()
matrice.var()